## Loading Data

---



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp '/content/gdrive/MyDrive/ML Project/archive.zip' /content/

In [3]:
!cd /content/
!unzip archive.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/JULIA/042.jpg     
  inflating: train/JULIA/043.jpg     
  inflating: train/JULIA/044.jpg     
  inflating: train/JULIA/045.jpg     
  inflating: train/JULIA/046.jpg     
  inflating: train/JULIA/047.jpg     
  inflating: train/JULIA/048.jpg     
  inflating: train/JULIA/049.jpg     
  inflating: train/JULIA/050.jpg     
  inflating: train/JULIA/051.jpg     
  inflating: train/JULIA/052.jpg     
  inflating: train/JULIA/053.jpg     
  inflating: train/JULIA/054.jpg     
  inflating: train/JULIA/055.jpg     
  inflating: train/JULIA/056.jpg     
  inflating: train/JULIA/057.jpg     
  inflating: train/JULIA/058.jpg     
  inflating: train/JULIA/059.jpg     
  inflating: train/JULIA/060.jpg     
  inflating: train/JULIA/061.jpg     
  inflating: train/JULIA/062.jpg     
  inflating: train/JULIA/063.jpg     
  inflating: train/JULIA/064.jpg     
  inflating: train/JULIA/065.jpg     
  inflating: train/JULIA/066.jpg     

## Main Code

---



In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
from sklearn.utils import shuffle
import tensorflow as tf

from tensorflow.keras.utils import image_dataset_from_directory

In [6]:
import matplotlib.pyplot as plt


def plot_metric(hist, metric):
    plt.plot(hist.history[f"{metric}"], label=f"{metric}")
    plt.plot(hist.history[f"val_{metric}"], label=f"val_{metric}")
    plt.title(f"{metric} Function Evolution")
    plt.legend()


def plot_hist(hist):
    plt.figure(figsize=(12, 8))
    plt.subplot(1, 2, 1)
    plot_metric(hist, 'loss')

    plt.subplot(1, 2, 2)
    plot_metric(hist, 'accuracy')

In [7]:
butterfly_path = '/content/'

In [8]:
train_ds = image_dataset_from_directory(butterfly_path + 'train')


Found 9285 files belonging to 75 classes.


In [9]:
valid_ds = image_dataset_from_directory(butterfly_path + 'valid')


Found 375 files belonging to 75 classes.


In [10]:
test_ds = image_dataset_from_directory(butterfly_path + 'test', shuffle=False)  # when shuffle is True, it auto shuffles on each iteration


Found 375 files belonging to 75 classes.


In [11]:
class_names = train_ds.class_names
class_count = len(class_names)
print(class_names)

['ADONIS', 'AFRICAN GIANT SWALLOWTAIL', 'AMERICAN SNOOT', 'AN 88', 'APPOLLO', 'ATALA', 'BANDED ORANGE HELICONIAN', 'BANDED PEACOCK', 'BECKERS WHITE', 'BLACK HAIRSTREAK', 'BLUE MORPHO', 'BLUE SPOTTED CROW', 'BROWN SIPROETA', 'CABBAGE WHITE', 'CAIRNS BIRDWING', 'CHECQUERED SKIPPER', 'CHESTNUT', 'CLEOPATRA', 'CLODIUS PARNASSIAN', 'CLOUDED SULPHUR', 'COMMON BANDED AWL', 'COMMON WOOD-NYMPH', 'COPPER TAIL', 'CRECENT', 'CRIMSON PATCH', 'DANAID EGGFLY', 'EASTERN COMA', 'EASTERN DAPPLE WHITE', 'EASTERN PINE ELFIN', 'ELBOWED PIERROT', 'GOLD BANDED', 'GREAT EGGFLY', 'GREAT JAY', 'GREEN CELLED CATTLEHEART', 'GREY HAIRSTREAK', 'INDRA SWALLOW', 'IPHICLUS SISTER', 'JULIA', 'LARGE MARBLE', 'MALACHITE', 'MANGROVE SKIPPER', 'MESTRA', 'METALMARK', 'MILBERTS TORTOISESHELL', 'MONARCH', 'MOURNING CLOAK', 'ORANGE OAKLEAF', 'ORANGE TIP', 'ORCHARD SWALLOW', 'PAINTED LADY', 'PAPER KITE', 'PEACOCK', 'PINE WHITE', 'PIPEVINE SWALLOW', 'POPINJAY', 'PURPLE HAIRSTREAK', 'PURPLISH COPPER', 'QUESTION MARK', 'RED ADMIRA

In [12]:
IMG_SIZE = 240
size = (IMG_SIZE, IMG_SIZE)
train_ds = train_ds.map(lambda image, label:
                        (tf.image.resize(image, size), label))
valid_ds = valid_ds.map(lambda image, label:
                        (tf.image.resize(image, size), label))
test_ds = test_ds.map(lambda image, label:
                      (tf.image.resize(image, size), label))


In [14]:
base_model = tf.keras.applications.efficientnet.EfficientNetB1(
    include_top=False,
    weights=None,
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling='max')
base_model.trainable = True
x = base_model.output
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(rate=.2)(x)
output = tf.keras.layers.Dense(class_count, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=output)
model.compile(tf.keras.optimizers.Adam(learning_rate=.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
hist = model.fit(train_ds, validation_data=valid_ds, epochs=10)
plot_hist(hist)

Epoch 1/10
 22/291 [=>............................] - ETA: 1:02:27 - loss: 5.5031 - accuracy: 0.0156

In [ ]:
!cd '/content'
!pwd

In [ ]:
model.save('efficient_10_epochs_scratch')

In [ ]:
!cp -r '/content/efficient_10_epochs_scratch' '/content/gdrive/MyDrive/ML Project'

In [ ]:
model.save('efficient_10_epochs_scratch.h5')

In [ ]:
!cp '/content/efficient_10_epochs_scratch.h5' '/content/gdrive/MyDrive/ML Project'

In [ ]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(test_ds, batch_size=5), axis=-1)
y_test = np.concatenate(
    [label for batch, label in test_ds.as_numpy_iterator()])
print(classification_report(y_test, y_pred))